In [83]:
import pandas as pd
import numpy as np

from sktime.forecasting.theta import ThetaForecaster

from util_desafio import log_retorno,calc_score

In [84]:
df = pd.read_csv("data/test/valores_ano.csv")
df.pop("Date")
df.head(2)

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,156.479996,18.75,236.779999,182.009995,135.419998,132.619995,366.290009,139.039993,407.209991,564.369995,...,87.769997,67.940002,63.540001,56.889999,116.889999,136.529999,125.368935,583.900024,64.239998,234.029999
1,151.190002,19.02,237.050003,179.699997,135.160004,131.360001,361.589996,135.770004,404.299988,554.000000,...,87.489998,68.370003,65.930000,57.180000,118.129997,137.259995,126.640778,587.599976,66.070000,225.119995


In [85]:
df.fillna(method="bfill", inplace=True)
df.fillna(method="ffill", inplace=True)

In [86]:
df_treino = df.iloc[:-20,:]
df_real = df.iloc[-20:,:]

print(df_treino.shape)
print(df_real.shape)

(178, 503)
(20, 503)


In [87]:
hyper_param_list = [1,5,7,10,15,20,25,30,35,40,50,60,70,80]
model_score = {}
horizon = 20

for col in df_treino.columns:
    aux_score = 100
    aux_hyper = 100

    for hyper_param in hyper_param_list:

        tforecaster = ThetaForecaster(sp=hyper_param)
        tforecaster.fit(df_treino[col])

        origem  = df_treino[col].shape[0]
        fh = np.arange(origem, origem + horizon)

        y_tpred = tforecaster.predict(fh)
        LR_previsto = log_retorno(pd.DataFrame(y_tpred).reset_index(drop=True), pd.DataFrame(df_treino[col].iloc[-20:]).reset_index(drop=True))
        LR_real = log_retorno(pd.DataFrame(df_real[col]).reset_index(drop=True), pd.DataFrame(df_treino[col].iloc[-20:]).reset_index(drop=True))

        score = calc_score(LR_previsto,LR_real)

        if aux_score > score:
            aux_score = score
            aux_hyper = hyper_param

    model_score[col] = [aux_hyper,aux_score]

model_score

{'A': [30, 0.037359614985266756],
 'AAL': [45, 0.09736863832493824],
 'AAP': [70, 0.10457117386158327],
 'AAPL': [55, 0.05646118557851561],
 'ABBV': [25, 0.02619744195356727],
 'ABC': [25, 0.031137772895294673],
 'ABMD': [75, 0.07333978275921207],
 'ABT': [65, 0.06580438512015827],
 'ACN': [50, 0.0889501483994502],
 'ADBE': [45, 0.16734378563641472],
 'ADI': [25, 0.048112617210643346],
 'ADM': [35, 0.04379059874760431],
 'ADP': [55, 0.03966943391089072],
 'ADSK': [45, 0.07757766394754288],
 'AEE': [80, 0.14175132434253712],
 'AEP': [80, 0.17612557676878404],
 'AES': [55, 0.08004367872181446],
 'AFL': [30, 0.04331073778023668],
 'AIG': [50, 0.051684773250013034],
 'AIZ': [35, 0.031186154735968807],
 'AJG': [55, 0.05150127505151284],
 'AKAM': [65, 0.10276334910606379],
 'ALB': [80, 0.09105967464858011],
 'ALGN': [65, 0.8699035780315207],
 'ALK': [45, 0.10375735774156225],
 'ALL': [45, 0.029300895232236956],
 'ALLE': [55, 0.14356179335623354],
 'AMAT': [45, 0.26670093991825183],
 'AMCR': 

In [ ]:
df_final = pd.DataFrame(columns=df_treino.columns, index=range(0,20))

df_final.index.names = ["Dia"]

for model in model_score:
    
    tforecaster = ThetaForecaster(sp=model_score[model][0])
    tforecaster.fit(df_treino[model])

    origem  = df_treino[model].shape[0]
    fh = np.arange(origem, origem + horizon)

    y_tpred = tforecaster.predict(fh)
    LR_previsto = log_retorno(pd.DataFrame(y_tpred).reset_index(drop=True), pd.DataFrame(df_treino[model].iloc[-20:]).reset_index(drop=True))
    df_final[model] = LR_previsto
        
df_final.sample(5)

In [91]:
df_final.to_csv("predicao.csv")

In [92]:
!python check_submission.py predicao.csv

Ok, você pode submeter o código
